In [23]:
import os
import sys
import re
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2016,10)

In [117]:
instructions = [
"value 5 goes to bot 2",
"bot 2 gives low to bot 1 and high to bot 0",
"value 3 goes to bot 1",
"bot 1 gives low to output 1 and high to bot 0",
"bot 0 gives low to output 2 and high to output 0",
"value 2 goes to bot 2"
]

In [59]:
# Initially, bot 1 starts with a value-3 chip, and bot 2 starts with a value-2 chip and a value-5 chip
state = {
    "bot" :{
        "1" : [3],
        "2" : [2,5],
        "10" : [9]
    },
    "output":{
        
    }
    
}

In [129]:
def new_bot():
    """
    simple function to initialize a new bot
    """
    return {
                "chips": [],
                "high": None,
                "low": None
            }
    
# regular expressions to match instructions
value_pattern = re.compile(r'value (\d+) goes to bot (\d+)')
gives_pattern = re.compile(r'bot (\d+) gives (low|high) to (\w+) (\d+)( and (low|high) to (\w+) (\d+))?')
def process_instruction(instruction):
    """
    Function to process instruction string

    parameters:
      - instruction: string

    returns:
      - None
      
    This function does not return a value, it updates the global state data structure
    """
    # is it a value instruction?
    match = value_pattern.match(instruction)
    if match:
        # initialize bot if it doesn't exist yet
        if not match.group(2) in state['bot']:
            state['bot'][match.group(2)] = new_bot()
        # give bot the new chip
        state['bot'][match.group(2)]['chips'].append(int(match.group(1)))
    else:
        # is it a give instruction
        match = gives_pattern.match(instruction)
        if match:
            #                  1-bot 2-chip 3-t1     4-t1idx                    6-chip   7-t2  8-t2idx 
            #match.groups() = ('1', 'low', 'output', '1', ' and high to bot 0', 'high', 'bot', '0')
            groups = match.groups()
            # give first chip
            give_chip(*groups[:4]) # ('1', 'low', 'output', '1')
            # is there a second chip?
            if not groups[6] is None:
                # give second chip
                give_chip(groups[0],*groups[5:]) # ('1','high', 'bot', '0')
        else:
            # we shouldn't get here, but who knows what is in the input
            print(f"Unhandled instruction: {instruction}")

def give_chip(source,highlow,target,target_idx):
    """
    Function to pass a chip from a bot to another bot or output bin

    parameters:
      - source: string, index of bot giving chip
      - highlow: string "high" or "low"
      - target: string, target type "bot" or "output"
      - target_idx: string, target index

    returns:
      - None
      
    This function does not return a value, it updates the global state data structure
    """
    # assigning bot to make this easier to read
    bot = state['bot'][source]
    # check high/low
    if highlow == 'high':
        # high, set chip to maximum value in bot, and remove that chip from bot
        chip = bot.pop(bot.index(max(bot)))
    else:
        # high, set chip to minumum value in bot, and remove that chip from bot
        chip = bot.pop(bot.index(min(bot)))
    # initialize target instance if it does not exist
    if target_idx not in state[target]:
        state[target][target_idx] = []
    # append chip to target
    state[target][target_idx].append(chip)
    
        

In [131]:
# Initially, bot 1 starts with a value-3 chip, and bot 2 starts with a value-2 chip and a value-5 chip
state = {
    "bot" :{
        "1" : [3],
        "2" : [2,5],
        "10" : [9]
    },
    "output":{
        
    }
    
}
for instruction in instructions:
    print(f"instruction = {instruction}")
    process_instruction(instruction)
    print(f"state = {state}")

instruction = value 5 goes to bot 2
state = {'bot': {'1': [3], '2': [2, 5, 5], '10': [9]}, 'output': {}}
instruction = bot 2 gives low to bot 1 and high to bot 0
state = {'bot': {'1': [3, 2], '2': [5], '10': [9], '0': [5]}, 'output': {}}
instruction = value 3 goes to bot 1
state = {'bot': {'1': [3, 2, 3], '2': [5], '10': [9], '0': [5]}, 'output': {}}
instruction = bot 1 gives low to output 1 and high to bot 0
state = {'bot': {'1': [3], '2': [5], '10': [9], '0': [5, 3]}, 'output': {'1': [2]}}
instruction = bot 0 gives low to output 2 and high to output 0
state = {'bot': {'1': [3], '2': [5], '10': [9], '0': []}, 'output': {'1': [2], '2': [3], '0': [5]}}
instruction = value 2 goes to bot 2
state = {'bot': {'1': [3], '2': [5, 2], '10': [9], '0': []}, 'output': {'1': [2], '2': [3], '0': [5]}}


In [101]:
my_list = [4,5,5]

In [35]:
max(my_list)

5

In [37]:
min(my_list)

4

In [103]:
tmp = my_list.pop(my_list.index(max(my_list)))
print(f"tmp = {tmp}")
print(f"my_list = {my_list}")


tmp = 5
my_list = [4, 5]


In [85]:
groups = ('1', 'low', 'output', '1', ' and high to bot 0', 'high', 'bot', '0')

In [89]:
groups[:4]

('1', 'low', 'output', '1')

In [95]:
groups[5:]

('high', 'bot', '0')

In [237]:
class Bin:
    """
    Simple class for output bins
    """
    def __init__(self,idx):
        """
        init Bin object
        """
        self.idx = idx
        self.type = 'output'
        # start empty
        self.chips = []

    def __str__(self):
        """
        String representation of bin
        """
        return f"output bin {self.idx} contains {self.chips}"

    def receive(self,chip):
        """
        Receive a chip from a bot
        """
        # add chip to chips, no other actions for output bins
        self.chips.append(chip)

class Bot:
    """
    class for bots
    """
    def __init__(self,idx):
        self.idx = idx
        self.type = 'bot'
        # chips empty
        self.chips = []
        self.comparisons = set()
        # placeholders for high and low targets
        self.targets = {
            "high": None,
            "low": None
        }

    def __str__(self):
        """
        String representation of bot
        """
        targets = {
            "high": None,
            "low": None
        }
        for high_low in ["high","low"]:
            if not self.targets[high_low] is None:
                targets[high_low] = f"{self.targets[high_low].type} {self.targets[high_low].idx}"
        retval =  f"""
        bot {self.idx} has chips {self.chips}
          sends high to {targets['high']}
          sends low to {targets['low']}
        """
        retval = f"bot {self.idx} {self.chips} comparisons: {self.comparisons}"
        return retval
    
    def receive(self,chip):
        """
        Receive a chip from a bot or input bin
        """
        # add chip to chips
        self.chips.append(chip)
        #print(f"bot {self.idx} received chip {chip}: [{self.chips}]")
        # need a trigger here for too many chips
        if len(self.chips) >= 2:
            print(f"{self.idx} comparing {self.chips}")
            self.comparisons.add(tuple(sorted(self.chips)))
            # distribute chips
            for high_low in ['high','low']:
                if not self.targets[high_low] is None:
                    self.give(self.targets[high_low],high_low)          

    def give(self,target,highlow):
        # give away a chip
        if highlow == 'high':
            # high, set chip to maximum value in bot, and remove that chip from bot
            chip = self.chips.pop(self.chips.index(max(self.chips)))
        else:
            # high, set chip to minumum value in bot, and remove that chip from bot
            chip = self.chips.pop(self.chips.index(min(self.chips)))
        target.receive(chip)
        
# regular expressions to match instructions
value_pattern = re.compile(r'value (\d+) goes to bot (\d+)')
gives_pattern = re.compile(r'bot (\d+) gives (low|high) to (\w+) (\d+)( and (low|high) to (\w+) (\d+))?')
def parse_instruction(instruction):
    """
    Function to parse instruction string, and update bots and bins

    parameters:
      - instruction: string

    returns:
      - None
      
    This function does not return a value, it updates the global state data structure
    """
    # is it a value instruction?
    match = value_pattern.match(instruction)
    if match:
        # initialize bot if it doesn't exist yet
        chip, idx = match.groups()
        if not idx in state['bot']:
            state['bot'][idx] = Bot(idx)
        # give bot the new chip
        state['bot'][idx].receive(int(chip))
    else:
        # is it a give instruction
        match = gives_pattern.match(instruction)
        if match:
            #                  1-bot 2-chip 3-t1     4-t1idx                    6-chip   7-t2  8-t2idx 
            #match.groups() = ('1', 'low', 'output', '1', ' and high to bot 0', 'high', 'bot', '0')
            groups = match.groups()
            # first chip rule
            idx = groups[0]
            
            # initialize bot if it doesn't exist
            if not idx in state['bot']:
                state['bot'][idx] = Bot(idx)
            bot = state['bot'][idx]
            for high_low,target_type, target_idx in (groups[1:4], groups[5:]): # ('1', 'low', 'output', '1')
                if high_low is None:
                    continue
                # initialize target if it doesn't exist
                if not target_idx in state[target_type]:
                    if target_type == 'bot':
                        state[target_type][target_idx] = Bot(target_idx)
                    else:
                        state[target_type][target_idx] = Bin(target_idx)
                # set target
                bot.targets[high_low] = state[target_type][target_idx]
        else:
            # we shouldn't get here, but who knows what is in the input
            print(f"Unhandled instruction: {instruction}")

In [239]:
state = {
    "bot": {},
    "output": {}
}

for instruction in sorted(instructions):
    #print(instruction)
    parse_instruction(instruction)


2 comparing [2, 5]
1 comparing [3, 2]
0 comparing [5, 3]


In [241]:
for group in ['bot','output']:
    for idx, inst in  state[group].items():
        print(inst)


bot 0 [] comparisons: {(3, 5)}
bot 1 [] comparisons: {(2, 3)}
bot 2 [] comparisons: {(2, 5)}
output bin 2 contains [3]
output bin 0 contains [5]
output bin 1 contains [2]
